In [64]:
from __future__ import print_function

import os
import cv2
import numpy as np
import pandas as pd
import tensorflow.keras
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score

from dataPreparation import loadData, preprocess

In [2]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

### test

In [3]:
SUBTRACT_PIXEL_MEAN = True
NUM_CLASSES = 7

In [4]:
valPath = os.path.join("./seti-data/primary_small/valid")
testPath = os.path.join("./seti-data/primary_small/test")

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

(xVal, yValStr) = loadData(valPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])
(xTest, yTestStr) = loadData(testPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])

In [5]:
le = preprocessing.LabelEncoder()
yVal = le.fit_transform(yValStr)
yTest = le.transform(yTestStr)

In [6]:
xVal = xVal.astype('float32') / 255
xTest = xTest.astype('float32') / 255

if SUBTRACT_PIXEL_MEAN:
    xTrainNMean = np.load('geekfile.npy')
    xVal -= xTrainNMean
    xTest -= xTrainNMean
    
yVal = tensorflow.keras.utils.to_categorical(yVal, NUM_CLASSES)
yTest = tensorflow.keras.utils.to_categorical(yTest, NUM_CLASSES)

In [7]:
import time

start_time = time.time()
new_model = tensorflow.keras.models.load_model("./model/my_model_89.h5")
elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Elapsed time: 0:01:22.95


In [8]:
scores = new_model.evaluate(xVal, yVal, verbose=0)
print(scores)

[0.43881555914878845, 0.8957143]


In [9]:
scores = new_model.evaluate(xTest, yTest, verbose=0)
print(scores)

[0.43807484690632137, 0.89]


### test based on kaggle kernel

In [73]:
predictions = new_model.predict(xTest, verbose = 0)

In [74]:
import sys
df = pd.DataFrame(predictions)
# df['truth'] = ''
df['truth'] = yTestStr
df['prediction_index'] = df[[0,1,2,3,4,5,6]].idxmax(axis=1)
df['prediction'] = le.inverse_transform(df.prediction_index)

In [75]:
df.head()

,0,1,2,3,4,5,6,truth,prediction_index,prediction
0,0.999326,1.199023e-06,6.470748e-07,1.992224e-06,6.577445e-04,2.450051e-07,0.000012,brightpixel,0,brightpixel
1,0.999523,9.197120e-07,2.432659e-07,3.335839e-04,1.970820e-07,2.020488e-05,0.000122,brightpixel,0,brightpixel
2,0.998158,2.204430e-07,3.751998e-08,6.360798e-10,1.834538e-03,7.255727e-07,0.000007,brightpixel,0,brightpixel
3,0.999547,1.241362e-06,8.348057e-06,2.801670e-06,3.493109e-04,9.475923e-06,0.000082,brightpixel,0,brightpixel
4,0.050795,1.310674e-03,1.983694e-02,8.827125e-01,4.279892e-02,2.594446e-04,0.002287,brightpixel,3,noise


In [76]:
cm = confusion_matrix(df['truth'], df['prediction'])
cm_df = pd.DataFrame(cm)
cm_df.columns = le.inverse_transform([0,1,2,3,4,5,6])
cm_df['signal'] = le.inverse_transform([0,1,2,3,4,5,6])

In [77]:
cm_df

,brightpixel,narrowband,narrowbanddrd,noise,squarepulsednarrowband,squiggle,squigglesquarepulsednarrowband,signal
0,78,0,0,21,0,0,1,brightpixel
1,0,99,1,0,0,0,0,narrowband
2,0,17,73,7,2,1,0,narrowbanddrd
3,2,0,0,94,1,1,2,noise
4,0,1,2,8,86,1,2,squarepulsednarrowband
5,0,1,0,1,0,98,0,squiggle
6,1,0,0,1,1,2,95,squigglesquarepulsednarrowband


In [78]:
accuracy = accuracy_score(df['truth'], df['prediction'])
accuracy

0.89